<a href="https://colab.research.google.com/github/gkuch22/ml-final/blob/main/final_model_experiment_patchTST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch # Main PyTorch Library
import matplotlib.pyplot as plt # Used for visualizing the images and plotting the training progress
import pandas as pd # Used to read/create dataframes (csv) and process tabular data
import numpy as np # preprocessing and numerical/mathematical operations
import seaborn as sns

device = "cuda" if torch.cuda.is_available() else "cpu" # detect the GPU if any, if not use CPU, change cuda to mps if you have a mac
print("Device available: ", device)


Device available:  cuda


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/cs231n/assignments/final/datasets/train.csv')
stores_df = pd.read_csv('/content/drive/MyDrive/cs231n/assignments/final/datasets/stores.csv')
features_df = pd.read_csv('/content/drive/MyDrive/cs231n/assignments/final/datasets/features.csv')
final_df = pd.read_csv('/content/drive/MyDrive/cs231n/assignments/final/datasets/test.csv')

In [ ]:
train_df

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False
...,...,...,...,...,...
421565,45,98,2012-09-28,508.37,False
421566,45,98,2012-10-05,628.10,False
421567,45,98,2012-10-12,1061.02,False
421568,45,98,2012-10-19,760.01,False


In [ ]:
def get_wmae(y_true, y_pred, weights):
  return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)

In [ ]:
!pip install neuralforecast -q

# MODEL PIPELINE

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class CustomPreprocessorClass(BaseEstimator, TransformerMixin):
  def __init__(self):
    pass

  def fit(self, X, y=None):
    return self

  def transform(self, X):
    df = X
    df['Date'] = pd.to_datetime(df['Date'])
    df['unique_id'] = df['Store'].astype(str) + '_' + df['Dept'].astype(str)
    df = df.sort_values(['unique_id', 'Date'])
    df = df.rename(columns={'Date': 'ds'})

    if 'Weekly_Sales' in df.columns:
      df = df.rename(columns={'Weekly_Sales': 'y'})
      col_names = ['unique_id', 'ds', 'y']
    else:
      df['y'] = np.nan
      col_names = ['unique_id', 'ds', 'y']

    df = df[col_names]

    return df

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from neuralforecast import NeuralForecast
from neuralforecast.models import PatchTST


class CustomPatchTSTClass(BaseEstimator, TransformerMixin):
  def __init__(self, H=100, INPUT_SIZE=75, MAX_STEPS=2500, LEARNING_RATE=1e-4, ENCODER_LAYERS=4, BATCH_SIZE=64, DROPOUT=0.3):
    self.H=H
    self.INPUT_SIZE=INPUT_SIZE
    self.MAX_STEPS=MAX_STEPS
    self.LEARNING_RATE=LEARNING_RATE
    self.ENCODER_LAYERS=ENCODER_LAYERS
    self.BATCH_SIZE=BATCH_SIZE
    self.DROPOUT=DROPOUT
    self.neural_forecast = None

  def fit(self, X, y=None):
    model = PatchTST(
        h=self.H,
        input_size=self.INPUT_SIZE,
        max_steps=self.MAX_STEPS,
        learning_rate=self.LEARNING_RATE,
        encoder_layers=self.ENCODER_LAYERS,
        batch_size=self.BATCH_SIZE,
        dropout=self.DROPOUT,
        )

    neural_forecast = NeuralForecast(models=[model], freq='W-FRI')
    neural_forecast.fit(df=X)

    self.neural_forecast = neural_forecast

    return self

  def transform(self, X):
    forecast_df = self.neural_forecast.predict()
    forecast_df = forecast_df[forecast_df['ds'].isin(X['ds'])]
    res_df = pd.merge(X, forecast_df, on=['unique_id', 'ds'], how='left')
    return res_df


In [ ]:
class CustomPatchTSTPipelineClass(BaseEstimator, TransformerMixin):
  def __init__(self):
    self.prep = CustomPreprocessorClass()
    self.model = CustomPatchTSTClass()

  def fit(self, train_df):
    train_df = self.prep.transform(train_df)
    self.model.fit(train_df)

  def predict(self, final_df):
    final_df = self.prep.transform(final_df)
    return self.model.transform(final_df)

In [ ]:
pipeline = CustomPatchTSTPipelineClass()
pipeline.fit(train_df)
precitions = pipeline.predict(final_df)

INFO:lightning_fabric.utilities.seed:Seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type              | Params | Mode 
-----------------------------------------------------------
0 | loss         | MAE               | 0      | train
1 | padder_train | ConstantPad1d     | 0      | train
2 | scaler       | TemporalNorm      | 0      | train
3 | model        | PatchTST_backbone | 648 K  | train
-----------------------------------------------------------
648 K     Trainable params
4         Non-trainable params
648 K     Total params
2.594     Total estimated model params size (MB)
115       Modules in train mode
0         Modules in 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=2500` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [ ]:
precitions = precitions.drop(columns=['y'])

In [ ]:
precitions['PatchTST'] = precitions['PatchTST'].fillna(0)

In [ ]:
precitions

,unique_id,ds,PatchTST
0,10_1,2012-11-02,67925.046875
1,10_1,2012-11-09,44971.558594
2,10_1,2012-11-16,39217.718750
3,10_1,2012-11-23,44840.160156
4,10_1,2012-11-30,50270.113281
...,...,...,...
115059,9_98,2013-01-04,9.649915
115060,9_99,2012-12-07,0.000000
115061,9_99,2013-07-05,0.000000
115062,9_99,2013-07-19,0.000000


In [ ]:
submission = precitions.copy()

In [ ]:
submission['Id'] = submission['unique_id'] + '_' + submission['ds'].astype(str)
submission = submission.rename(columns={'PatchTST': 'Weekly_Sales'})
submission = submission[['Id', 'Weekly_Sales']].reset_index(drop=True)

In [ ]:
submission

,Id,Weekly_Sales
0,10_1_2012-11-02,67925.046875
1,10_1_2012-11-09,44971.558594
2,10_1_2012-11-16,39217.718750
3,10_1_2012-11-23,44840.160156
4,10_1_2012-11-30,50270.113281
...,...,...
115059,9_98_2013-01-04,9.649915
115060,9_99_2012-12-07,0.000000
115061,9_99_2013-07-05,0.000000
115062,9_99_2013-07-19,0.000000


In [ ]:
submission.to_csv('patchtst_submission.csv', index=False)

# WITHOUT PIPELINE

In [ ]:
train_df['Date'] = pd.to_datetime(train_df['Date'])
merged_df = train_df

In [ ]:
merged_df

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False
...,...,...,...,...,...
421565,45,98,2012-09-28,508.37,False
421566,45,98,2012-10-05,628.10,False
421567,45,98,2012-10-12,1061.02,False
421568,45,98,2012-10-19,760.01,False


In [ ]:
merged_df['unique_id'] = merged_df['Store'].astype(str) + '_' + merged_df['Dept'].astype(str)
merged_df = merged_df.sort_values(['unique_id', 'Date'])

In [ ]:
merged_df['Year'] = merged_df['Date'].dt.year
merged_df['Month'] = merged_df['Date'].dt.month
merged_df['Week'] = merged_df['Date'].dt.isocalendar().week.astype(int)

merged_df = merged_df.rename(columns={'Date': 'ds', 'Weekly_Sales': 'y'})

In [ ]:
year_split = 2012
month_split = 6

train_df = merged_df[merged_df['Year'] < year_split]
valid_df = merged_df[(merged_df['Year'] == year_split) & (merged_df['Month'] < month_split)]
test_df = merged_df[(merged_df['Year'] == year_split) & (merged_df['Month'] >= month_split)]

In [ ]:
train_df

,Store,Dept,ds,y,IsHoliday,unique_id,Year,Month,Week
87524,10,1,2010-02-05,40212.84,False,10_1,2010,2,5
87525,10,1,2010-02-12,67699.32,True,10_1,2010,2,6
87526,10,1,2010-02-19,49748.33,False,10_1,2010,2,7
87527,10,1,2010-02-26,33601.22,False,10_1,2010,2,8
87528,10,1,2010-03-05,36572.44,False,10_1,2010,3,9
...,...,...,...,...,...,...,...,...,...
87516,9,98,2011-11-25,60.75,True,9_98,2011,11,47
87517,9,98,2011-12-02,54.75,False,9_98,2011,12,48
87518,9,98,2011-12-09,75.77,False,9_98,2011,12,49
87519,9,98,2011-12-16,66.75,False,9_98,2011,12,50


In [ ]:
weights_valid = valid_df[['unique_id', 'ds', 'IsHoliday']]

In [ ]:
store_dept_pair = train_df[(merged_df['Store'] == 1) & (merged_df['Dept'] == 1)].copy()
print(store_dept_pair.shape)

store_dept_pair = valid_df[(merged_df['Store'] == 1) & (merged_df['Dept'] == 1)].copy()
print(store_dept_pair.shape)

(100, 9)
(21, 9)


/tmp/ipython-input-13-1212814644.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  store_dept_pair = train_df[(merged_df['Store'] == 1) & (merged_df['Dept'] == 1)].copy()
/tmp/ipython-input-13-1212814644.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  store_dept_pair = valid_df[(merged_df['Store'] == 1) & (merged_df['Dept'] == 1)].copy()


In [ ]:
col_names = ['unique_id', 'ds', 'y']
train_df = train_df[col_names]
valid_df = valid_df[col_names]
test_df  = test_df[col_names]

In [ ]:
train_df['ds'].min(), train_df['ds'].max(), valid_df['ds'].min(), valid_df['ds'].max(), test_df['ds'].min(), test_df['ds'].max()

(Timestamp('2010-02-05 00:00:00'),
 Timestamp('2011-12-30 00:00:00'),
 Timestamp('2012-01-06 00:00:00'),
 Timestamp('2012-05-25 00:00:00'),
 Timestamp('2012-06-01 00:00:00'),
 Timestamp('2012-10-26 00:00:00'))

In [ ]:
train_df

,unique_id,ds,y
87524,10_1,2010-02-05,40212.84
87525,10_1,2010-02-12,67699.32
87526,10_1,2010-02-19,49748.33
87527,10_1,2010-02-26,33601.22
87528,10_1,2010-03-05,36572.44
...,...,...,...
87516,9_98,2011-11-25,60.75
87517,9_98,2011-12-02,54.75
87518,9_98,2011-12-09,75.77
87519,9_98,2011-12-16,66.75


In [ ]:
def get_wmae(y_true, y_pred, weights):
  return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)

In [ ]:
H=21
INPUT_SIZE=75
MAX_STEPS=2500
LEARNING_RATE=1e-4
ENCODER_LAYERS=4
BATCH_SIZE=64
DROPOUT=0.3

In [ ]:
# from neuralforecast import NeuralForecast
# from neuralforecast.models import PatchTST

# model = PatchTST(
#     h=H,
#     input_size=INPUT_SIZE,
#     max_steps=MAX_STEPS,
#     learning_rate=LEARNING_RATE,
#     encoder_layers=ENCODER_LAYERS,
#     batch_size=BATCH_SIZE,
#     dropout=DROPOUT,
#     )

# neural_forecast = NeuralForecast(models=[model], freq='W-FRI')
# neural_forecast.fit(df=train_df)

# valid_forecast_df = neural_forecast.predict()


INFO:lightning_fabric.utilities.seed:Seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type              | Params | Mode 
-----------------------------------------------------------
0 | loss         | MAE               | 0      | train
1 | padder_train | ConstantPad1d     | 0      | train
2 | scaler       | TemporalNorm      | 0      | train
3 | model        | PatchTST_backbone | 557 K  | train
-----------------------------------------------------------
557 K     Trainable params
4         Non-trainable params
557 K     Total params
2.230     Total estimated model params size (MB)
115       Modules in train mode
0         Modules in 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=2500` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [ ]:
valid_forecast_df

,unique_id,ds,PatchTST
0,10_1,2012-01-06,38045.109375
1,10_1,2012-01-13,33921.464844
2,10_1,2012-01-20,42664.242188
3,10_1,2012-01-27,32164.755859
4,10_1,2012-02-03,43318.472656
...,...,...,...
69337,9_98,2012-04-20,18.887556
69338,9_98,2012-04-27,20.049030
69339,9_98,2012-05-04,16.287489
69340,9_98,2012-05-11,17.104406


In [ ]:
resvalid_df = pd.merge(valid_forecast_df, valid_df, on=['unique_id', 'ds'], how='inner')

In [ ]:
resvalid_df = pd.merge(resvalid_df, weights_valid, on=['unique_id', 'ds'], how='left')

In [ ]:
resvalid_df['weight'] = np.where(resvalid_df['IsHoliday'], 5, 1)

In [ ]:
valid_wmae = get_wmae(y_true=resvalid_df['y'], y_pred=resvalid_df['PatchTST'], weights=resvalid_df['weight'])
print(f"valid wmae: {valid_wmae:.2f}")
print()

valid wmae: 1983.28



In [ ]:
resvalid_df

,unique_id,ds,PatchTST,y,IsHoliday,weight
0,10_1,2012-01-06,38045.109375,28520.49,False,1
1,10_1,2012-01-13,33921.464844,30107.31,False,1
2,10_1,2012-01-20,42664.242188,31180.23,False,1
3,10_1,2012-01-27,32164.755859,32559.13,False,1
4,10_1,2012-02-03,43318.472656,36444.00,False,1
...,...,...,...,...,...,...
62075,9_96,2012-05-18,3853.768066,4397.05,False,1
62076,9_96,2012-05-25,4229.556641,4376.16,False,1
62077,9_98,2012-01-06,34.247646,0.50,False,1
62078,9_98,2012-01-27,30.589334,-1.00,False,1


# MLFLOW TRACKING

In [ ]:
# !pip install dagshub mlflow -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
# import dagshub
# import mlflow

# dagshub.init(repo_owner='gkuch22', repo_name='ml-final', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=da3ea722-d10a-4393-ae1e-24578657c864&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=e2934170627235914f3e76f9962320d43a793b8f0897efd76cfecd33a8f36f8e




Accessing as gkuch22

Initialized MLflow to track repo "gkuch22/ml-final"

Repository gkuch22/ml-final initialized!

In [ ]:
# experiment_name = "PatchTST_train"
# run_name = "max_steps++input_size++"

In [ ]:
# import mlflow

# mlflow.set_experiment(experiment_name)

# with mlflow.start_run(run_name=run_name):

#     mlflow.log_param("model_type", "PatchTST")
#     mlflow.log_param("h", H)
#     mlflow.log_param("input_size", INPUT_SIZE)
#     mlflow.log_param("max_steps", MAX_STEPS)
#     mlflow.log_param("learning_rate", LEARNING_RATE)
#     mlflow.log_param("encoder_layers", ENCODER_LAYERS)
#     mlflow.log_param("batch_size", BATCH_SIZE)
#     mlflow.log_param("dropout", DROPOUT)


#     # model = Prophet()
#     # model.fit(train_df[['ds', 'y']])

#     # y_pred_train = model.predict(train_df[['ds']])
#     # y_pred_valid = model.predict(valid_df[['ds']])

#     # yhat_train = y_pred_train['yhat'].values
#     # yhat_valid = y_pred_valid['yhat'].values

#     # weights_train = np.where(train_df['IsHoliday'], 5, 1)
#     # weights_valid = np.where(valid_df['IsHoliday'], 5, 1)

#     # train_wmae = get_wmae(train_df['y'].values, yhat_train, weights_train)
#     # valid_wmae = get_wmae(valid_df['y'].values, yhat_valid, weights_valid)

#     # print(f"train wmae: {train_wmae:.2f}")
#     # print(f"valid wmae: {valid_wmae:.2f}")
#     # print()

#     # mlflow.log_metric("train_wmae", train_wmae)
#     mlflow.log_metric("valid_wmae", valid_wmae)

#     # mlflow.sklearn.log_model(pipeline, "model")


🏃 View run max_steps++input_size++ at: https://dagshub.com/gkuch22/ml-final.mlflow/#/experiments/7/runs/a4b76cbb9ec74e03b05cae53ee3c6d96
🧪 View experiment at: https://dagshub.com/gkuch22/ml-final.mlflow/#/experiments/7
